In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='7'
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
from tqdm import tqdm
FLAGS = edict()

# FLAGS.weights ='./checkpoints/day_tw_qat_tf23/save_model_final_tflite/'
FLAGS.weights ='./checkpoints/yolov4_tiny/save_model_0000_tflite/'
FLAGS.output ='./checkpoints/tmp.tflite'
FLAGS.input_size =608
FLAGS.quantize_mode ='float32'
FLAGS.dataset ="datasets/data_selection_mix/anno/val_3cls.txt"


2022-07-06 16:19:41.258414: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


file /home/user/anaconda3/envs/WJtf29/lib/python3.8/site-packages/tensorflow/lite/python/interpreter is not end with tflite_runtime/interpreter


In [2]:
def representative_data_gen():
  len_img=10
  fimage = open(FLAGS.dataset).readlines()
  fimage = [line.split()[0] for line in fimage]
  np.random.seed(0)
  np.random.shuffle(fimage)
  with tqdm(total=len_img, ncols=200) as pbar:
    for input_value in range(len_img):
      if os.path.exists(fimage[input_value]):
        original_image=cv2.imread(fimage[input_value])
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        # Processing V1
        # image_data = utils.image_preprocess(np.copy(original_image), [FLAGS.input_size, FLAGS.input_size])
        #####################################################################################################
        # Processing V2
        image_data = cv2.resize(np.copy(original_image), (FLAGS.input_size, FLAGS.input_size))
        image_data = image_data / 255.0
        #####################################################################################################
        img_in = image_data[np.newaxis, ...].astype(np.float32)
        pbar.set_postfix({
          'image': fimage[input_value]
        })
        pbar.update(1)
        yield [img_in]
      else:
        pbar.set_postfix({
          'image': ''
        })
        pbar.update(1)

def add_statistic(filename):
  layer_stats = pd.read_csv(filename)
  print(layer_stats.head())
  layer_stats['range'] = 255.0 * layer_stats['scale']
  layer_stats['rmse/scale'] = layer_stats.apply(
      lambda row: np.sqrt(row['mean_squared_error']) / row['scale'], axis=1)
  return layer_stats
  # layer_stats[['op_name', 'range', 'rmse/scale']].head()
  

In [3]:
if not os.path.exists(os.path.dirname(FLAGS.output)):
  os.makedirs(os.path.dirname(FLAGS.output))
converter = tf.lite.TFLiteConverter.from_saved_model(FLAGS.weights)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
debugger = tf.lite.experimental.QuantizationDebugger(
  converter=converter, debug_dataset=representative_data_gen)
debugger.run()


origin_int8_model = debugger.get_nondebug_quantized_model()
num_of_bytes = open('./tflite_exp/origin_int8_model.tflite', 'wb').write(origin_int8_model)
print(f'origin model {num_of_bytes} bytes')

RESULTS_FILE = './debugger_results.csv'
RESULTS_FILE_V2 = './debugger_results_V2.csv'
with open(RESULTS_FILE, 'w') as f:
  debugger.layer_statistics_dump(f)
layer_stats = add_statistic(RESULTS_FILE)
layer_stats.to_csv(RESULTS_FILE_V2)


2022-07-06 16:21:49.345332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-06 16:21:50.217841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7665 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:8c:00.0, compute capability: 7.5
2022-07-06 16:21:56.262922: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-07-06 16:21:56.266067: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-07-06 16:21:56.267547: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./checkpoints/yolov4

origin model 6014016 bytes
   op_name  tensor_idx  num_elements    stddev  mean_error  max_abs_error  \
0     RELU         119     2957312.0  0.025487   -0.006334       0.063999   
1      PAD         123     2976800.0  0.000000    0.000000       0.000000   
2  CONV_2D         127     1478656.0  0.164274    0.001327      41.843178   
3     RELU         131     1478656.0  0.033161   -0.000096       0.081139   
4  CONV_2D         135     1478656.0  0.145927    0.000984      45.764008   

   mean_squared_error     scale  zero_point  \
0            0.000690  0.131196        -128   
1            0.000000  0.131196        -128   
2            0.027372  0.376181          17   
3            0.001100  0.162274        -128   
4            0.021484  0.267913          48   

                                         tensor_name  
0                              model/tf.nn.relu/Relu  
1                   model/quant_zero_padding2d_1/Pad  
2  model/quant_batch_normalization_1/FusedBatchNo...  
3      

0, inference_type: 6, input_inference_type: 0, output_inference_type: 0


In [7]:
import subprocess
# last_k=85
st=100
end=len(list(layer_stats['tensor_name']))
for last_k in range(1):
    suspected_layers = list(layer_stats['tensor_name'])[st:end]
    debug_options = tf.lite.experimental.QuantizationDebugOptions(
        denylisted_nodes=suspected_layers)
    debugger = tf.lite.experimental.QuantizationDebugger(
        converter=converter,
        debug_dataset=representative_data_gen,
        debug_options=debug_options)

    filename = f'./tflite_exp/selective_int8_model_rev_{len(suspected_layers)}layer_st{st}_end{end}.tflite'
    selective_quantized_model_dbg = debugger.get_nondebug_quantized_model()
    with open(filename, 'wb') as f:
        num_of_bytes = f.write(selective_quantized_model_dbg)
    f.close()
    print(f'selective model {num_of_bytes} bytes. {len(suspected_layers)} layers')
    # subprocess.run(['python', 'evaluate_map_v3.py', '--weights', filename, '--framework', 'tflite'])

2022-07-06 16:33:58.602510: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-07-06 16:33:58.602565: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-07-06 16:33:58.602955: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./checkpoints/yolov4_tiny/save_model_0000_tflite/
2022-07-06 16:33:58.654848: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-07-06 16:33:58.654945: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: ./checkpoints/yolov4_tiny/save_model_0000_tflite/
2022-07-06 16:33:58.776680: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-06 16:33:59.364534: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: ./checkpoints/yolov4_tiny/save_model_0000_tflite/
2022-07-06 16:33:59.618965: I tensorflow/

selective model 6023072 bytes. 21 layers


fully_quantize: 0, inference_type: 6, input_inference_type: 0, output_inference_type: 0
